In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
df = pd.read_csv("../input/train.csv")
df.head()

,ID,Age,Worker Class,IC,OC,Schooling,Timely Income,Enrolled,Married_Life,MIC,MOC,Cast,Hispanic,Sex,MLU,Reason,Full/Part,Gain,Loss,Stock,Tax Status,Area,State,Detailed,Summary,Weight,MSA,REG,MOVE,Live,PREV,NOP,Teen,COB FATHER,COB MOTHER,COB SELF,Citizen,Own/Self,Fill,Vet_Benefits,Weaks,WorkingPeriod,Class
0,149068,7,?,0,0,Edu4,0,?,MS3,?,?,TypeD,HA,M,?,?,FB,0,0,0,J1,?,?,D4,sum4,2253.40,StatusA,StatusA,StatusA,YES,?,0,B,c24,c24,c24,Case1,0,?,0,0,94,0
1,174234,40,WC3,43,26,Edu1,0,?,MS4,MIC_D,MOC_E,TypeA,HA,F,?,?,FC,0,0,0,J3,?,?,D5,sum5,2802.99,?,?,?,?,?,6,?,c24,c24,c24,Case1,0,?,2,52,95,0
2,70782,25,WC2,33,19,Edu5,0,?,MS4,MIC_N,MOC_H,TypeA,HA,F,?,?,FB,0,0,0,J3,?,?,D5,sum5,2831.02,StatusA,StatusA,StatusA,YES,?,2,?,c24,c24,c24,Case1,2,?,2,48,94,0
3,5542,15,?,0,0,Edu16,0,?,MS3,?,?,TypeA,HA,F,?,?,FB,0,0,0,J1,?,?,D4,sum4,1668.38,StatusA,StatusA,StatusA,YES,?,0,B,c24,c24,c24,Case1,0,?,2,0,94,0
4,126542,48,WC2,33,29,Edu16,0,?,MS4,MIC_N,MOC_G,TypeB,HA,M,?,?,FC,0,0,0,J3,?,?,D2,sum2,1061.32,?,?,?,?,?,2,?,c23,c23,c23,Case3,0,?,2,52,95,0


In [3]:
to_keep = ['ID', 'Age', 'IC', 'OC', 'Schooling', 'Timely Income',
       'Married_Life', 'Cast', 'Hispanic', 'Sex',
       'Full/Part', 'Gain', 'Loss', 'Stock', 'Tax Status',
       'Summary', 'Weight',
       'NOP',
       'Citizen', 'Own/Self', 'Vet_Benefits', 'Weaks', 'WorkingPeriod',
       'Class']

In [4]:
for i in df.columns:
  print(str(i) + " " + str(len(df[i].unique())))

ID 100000
Age 91
Worker Class 9
IC 52
OC 47
Schooling 17
Timely Income 887
Enrolled 3
Married_Life 7
MIC 24
MOC 15
Cast 5
Hispanic 10
Sex 2
MLU 3
Reason 6
Full/Part 8
Gain 129
Loss 107
Stock 1134
Tax Status 6
Area 6
State 50
Detailed 37
Summary 8
Weight 64567
MSA 9
REG 8
MOVE 9
Live 3
PREV 3
NOP 7
Teen 5
COB FATHER 43
COB MOTHER 43
COB SELF 43
Citizen 5
Own/Self 3
Fill 3
Vet_Benefits 3
Weaks 53
WorkingPeriod 2
Class 2


In [5]:
df2 = df[to_keep]

In [6]:
df2.head()

,ID,Age,IC,OC,Schooling,Timely Income,Married_Life,Cast,Hispanic,Sex,Full/Part,Gain,Loss,Stock,Tax Status,Summary,Weight,NOP,Citizen,Own/Self,Vet_Benefits,Weaks,WorkingPeriod,Class
0,149068,7,0,0,Edu4,0,MS3,TypeD,HA,M,FB,0,0,0,J1,sum4,2253.40,0,Case1,0,0,0,94,0
1,174234,40,43,26,Edu1,0,MS4,TypeA,HA,F,FC,0,0,0,J3,sum5,2802.99,6,Case1,0,2,52,95,0
2,70782,25,33,19,Edu5,0,MS4,TypeA,HA,F,FB,0,0,0,J3,sum5,2831.02,2,Case1,2,2,48,94,0
3,5542,15,0,0,Edu16,0,MS3,TypeA,HA,F,FB,0,0,0,J1,sum4,1668.38,0,Case1,0,2,0,94,0
4,126542,48,33,29,Edu16,0,MS4,TypeB,HA,M,FC,0,0,0,J3,sum2,1061.32,2,Case3,0,2,52,95,0


In [7]:
df2 = df2.replace({"?": np.nan})

In [8]:
df2 = df2.fillna(df.mode().iloc[0])

In [9]:
df2.isnull().values.any()

False

In [10]:
df2.head()

,ID,Age,IC,OC,Schooling,Timely Income,Married_Life,Cast,Hispanic,Sex,Full/Part,Gain,Loss,Stock,Tax Status,Summary,Weight,NOP,Citizen,Own/Self,Vet_Benefits,Weaks,WorkingPeriod,Class
0,149068,7,0,0,Edu4,0,MS3,TypeD,HA,M,FB,0,0,0,J1,sum4,2253.40,0,Case1,0,0,0,94,0
1,174234,40,43,26,Edu1,0,MS4,TypeA,HA,F,FC,0,0,0,J3,sum5,2802.99,6,Case1,0,2,52,95,0
2,70782,25,33,19,Edu5,0,MS4,TypeA,HA,F,FB,0,0,0,J3,sum5,2831.02,2,Case1,2,2,48,94,0
3,5542,15,0,0,Edu16,0,MS3,TypeA,HA,F,FB,0,0,0,J1,sum4,1668.38,0,Case1,0,2,0,94,0
4,126542,48,33,29,Edu16,0,MS4,TypeB,HA,M,FC,0,0,0,J3,sum2,1061.32,2,Case3,0,2,52,95,0


In [11]:
y=df2["Class"]
x=df2.drop(columns=["ID", "Class"])

In [12]:
x_new = pd.get_dummies(x, columns=['Schooling', 'Married_Life', 'Cast',
        'Hispanic', 'Sex', 'Full/Part', 'Tax Status',
        'Summary', 'NOP', 'Citizen', 'Own/Self', 'Vet_Benefits',])

In [13]:
cols = x_new.columns

In [14]:
x_new.head()

,Age,IC,OC,Timely Income,Gain,Loss,Stock,Weight,Weaks,WorkingPeriod,Schooling_Edu1,Schooling_Edu10,Schooling_Edu11,Schooling_Edu12,Schooling_Edu13,Schooling_Edu14,Schooling_Edu15,Schooling_Edu16,Schooling_Edu17,Schooling_Edu2,Schooling_Edu3,Schooling_Edu4,Schooling_Edu5,Schooling_Edu6,Schooling_Edu7,Schooling_Edu8,Schooling_Edu9,Married_Life_MS1,Married_Life_MS2,Married_Life_MS3,Married_Life_MS4,Married_Life_MS5,Married_Life_MS6,Married_Life_MS7,Cast_TypeA,Cast_TypeB,Cast_TypeC,Cast_TypeD,Cast_TypeE,Hispanic_HA,Hispanic_HB,Hispanic_HC,Hispanic_HD,Hispanic_HE,Hispanic_HF,Hispanic_HG,Hispanic_HI,Hispanic_HJ,Sex_F,Sex_M,Full/Part_FA,Full/Part_FB,Full/Part_FC,Full/Part_FD,Full/Part_FE,Full/Part_FF,Full/Part_FG,Full/Part_FH,Tax Status_J1,Tax Status_J2,Tax Status_J3,Tax Status_J4,Tax Status_J5,Tax Status_J6,Summary_sum1,Summary_sum2,Summary_sum3,Summary_sum4,Summary_sum5,Summary_sum6,Summary_sum7,Summary_sum8,NOP_0,NOP_1,NOP_2,NOP_3,NOP_4,NOP_5,NOP_6,Citizen_Case1,Citizen_Case2,Citizen_Case3,Citizen_Case4,Citizen_Case5,Own/Self_0,Own/Self_1,Own/Self_2,Vet_Benefits_0,Vet_Benefits_1,Vet_Benefits_2
0,7,0,0,0,0,0,0,2253.40,0,94,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
1,40,43,26,0,0,0,0,2802.99,52,95,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1
2,25,33,19,0,0,0,0,2831.02,48,94,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,15,0,0,0,0,0,0,1668.38,0,94,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,48,33,29,0,0,0,0,1061.32,52,95,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1


In [15]:
abc = AdaBoostClassifier(random_state=42)
abc = abc.fit(x_new,y)

In [16]:
f_importances = abc.feature_importances_

In [17]:
print(f_importances)

[0.14 0.08 0.12 0.04 0.08 0.02 0.08 0.   0.08 0.   0.02 0.   0.   0.02
 0.02 0.02 0.02 0.02 0.02 0.   0.02 0.   0.02 0.02 0.   0.   0.02 0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.02 0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.02 0.   0.   0.   0.   0.   0.
 0.   0.   0.02 0.   0.   0.   0.02 0.   0.   0.   0.02 0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.02 0.   0.   0.   0.   0.
 0.   0.02 0.   0.   0.   0.  ]


In [18]:
imp_index = np.argsort(f_importances)

In [19]:
print(imp_index)

[44 63 61 60 59 57 56 55 54 53 52 51 50 48 47 46 45 88 64 43 65 68 87 86
 84 83 82 81 80 79 77 76 75 74 73 72 71 70 69 67 42 89 40 19 12 41 21 11
  9 24 25  7 28 29 27 31 32 33 34 35 36 37 38 30 14 78  5 10 85 13 15 20
 17 39 49 16 58 26 62 22 66 18 23  3  8  6  4  1  2  0]


In [20]:
print([cols[i] for i in imp_index])

['Hispanic_HF', 'Tax Status_J6', 'Tax Status_J4', 'Tax Status_J3', 'Tax Status_J2', 'Full/Part_FH', 'Full/Part_FG', 'Full/Part_FF', 'Full/Part_FE', 'Full/Part_FD', 'Full/Part_FC', 'Full/Part_FB', 'Full/Part_FA', 'Sex_F', 'Hispanic_HJ', 'Hispanic_HI', 'Hispanic_HG', 'Vet_Benefits_1', 'Summary_sum1', 'Hispanic_HE', 'Summary_sum2', 'Summary_sum5', 'Vet_Benefits_0', 'Own/Self_2', 'Own/Self_0', 'Citizen_Case5', 'Citizen_Case4', 'Citizen_Case3', 'Citizen_Case2', 'Citizen_Case1', 'NOP_5', 'NOP_4', 'NOP_3', 'NOP_2', 'NOP_1', 'NOP_0', 'Summary_sum8', 'Summary_sum7', 'Summary_sum6', 'Summary_sum4', 'Hispanic_HD', 'Vet_Benefits_2', 'Hispanic_HB', 'Schooling_Edu2', 'Schooling_Edu11', 'Hispanic_HC', 'Schooling_Edu4', 'Schooling_Edu10', 'WorkingPeriod', 'Schooling_Edu7', 'Schooling_Edu8', 'Weight', 'Married_Life_MS2', 'Married_Life_MS3', 'Married_Life_MS1', 'Married_Life_MS5', 'Married_Life_MS6', 'Married_Life_MS7', 'Cast_TypeA', 'Cast_TypeB', 'Cast_TypeC', 'Cast_TypeD', 'Cast_TypeE', 'Married_Life_

In [21]:
sel_features = cols[imp_index[-30:]]
print(str(sel_features))
X_train = x_new[[cols for cols in sel_features]]

Index(['Cast_TypeC', 'Cast_TypeD', 'Cast_TypeE', 'Married_Life_MS4',
       'Schooling_Edu13', 'NOP_6', 'Loss', 'Schooling_Edu1', 'Own/Self_1',
       'Schooling_Edu12', 'Schooling_Edu14', 'Schooling_Edu3',
       'Schooling_Edu16', 'Hispanic_HA', 'Sex_M', 'Schooling_Edu15',
       'Tax Status_J1', 'Schooling_Edu9', 'Tax Status_J5', 'Schooling_Edu5',
       'Summary_sum3', 'Schooling_Edu17', 'Schooling_Edu6', 'Timely Income',
       'Weaks', 'Stock', 'Gain', 'IC', 'OC', 'Age'],
      dtype='object')


In [22]:
X_train.head()

,Cast_TypeC,Cast_TypeD,Cast_TypeE,Married_Life_MS4,Schooling_Edu13,NOP_6,Loss,Schooling_Edu1,Own/Self_1,Schooling_Edu12,Schooling_Edu14,Schooling_Edu3,Schooling_Edu16,Hispanic_HA,Sex_M,Schooling_Edu15,Tax Status_J1,Schooling_Edu9,Tax Status_J5,Schooling_Edu5,Summary_sum3,Schooling_Edu17,Schooling_Edu6,Timely Income,Weaks,Stock,Gain,IC,OC,Age
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,7
1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,52,0,0,43,26,40
2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,48,0,0,33,19,25
3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,15
4,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,52,0,0,33,29,48


In [23]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

In [24]:
ros = RandomOverSampler(random_state=42)
X_r, y_r = ros.fit_resample(X_train, y_train)

In [25]:
abc2 = AdaBoostClassifier(n_estimators=500, random_state = 42)
abc2.fit(X_r,y_r)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=500, random_state=42)

In [26]:
y_pred_val = abc2.predict(X_val)

In [27]:
roc_auc_score(y_val,y_pred_val)

0.8745572655141658

In [28]:
#pd.set_option('display.max_columns', 500)
df_test = pd.read_csv("../input/test.csv")
df_test.head()

,ID,Age,Worker Class,IC,OC,Schooling,Timely Income,Enrolled,Married_Life,MIC,MOC,Cast,Hispanic,Sex,MLU,Reason,Full/Part,Gain,Loss,Stock,Tax Status,Area,State,Detailed,Summary,Weight,MSA,REG,MOVE,Live,PREV,NOP,Teen,COB FATHER,COB MOTHER,COB SELF,Citizen,Own/Self,Fill,Vet_Benefits,Weaks,WorkingPeriod
0,32701,57,WC1,1,44,Edu9,0,?,MS5,MIC_U,MOC_L,TypeA,HE,M,?,?,FB,0,0,0,J4,?,?,D8,sum2,2087.06,StatusA,StatusA,StatusA,YES,?,1,?,c24,c24,c24,Case1,0,?,2,52,94
1,105694,6,?,0,0,Edu4,0,?,MS3,?,?,TypeA,HA,F,?,?,FB,0,0,0,J1,?,?,D4,sum4,979.61,StatusA,StatusA,StatusA,YES,?,0,F,c24,c24,c24,Case1,0,?,0,0,94
2,139722,16,WC2,43,31,Edu14,0,Uni1,MS3,MIC_D,MOC_G,TypeD,HA,F,?,?,FA,0,0,0,J1,?,?,D9,sum1,2993.58,?,?,?,?,?,0,N,c24,c24,c24,Case1,0,?,2,0,95
3,196082,56,?,0,0,Edu16,0,?,MS2,?,?,TypeA,HA,F,?,?,FA,0,0,0,J1,?,?,D8,sum2,2216.33,?,?,?,?,?,0,?,c24,c24,c24,Case1,0,?,2,0,95
4,82783,6,?,0,0,Edu4,0,?,MS3,?,?,TypeA,HA,M,?,?,FB,0,0,0,J1,?,?,D4,sum4,429.67,?,?,?,?,?,0,B,c27,c24,c24,Case1,0,?,0,0,95


In [29]:
to_keep2 = ['ID', 'Age', 'IC', 'OC', 'Schooling', 'Timely Income',
       'Married_Life', 'Cast', 'Hispanic', 'Sex',
       'Full/Part', 'Gain', 'Loss', 'Stock', 'Tax Status',
       'Summary', 'Weight',
       'NOP',
       'Citizen', 'Own/Self', 'Vet_Benefits', 'Weaks', 'WorkingPeriod']

In [30]:
dft2 = df_test[to_keep2]

In [31]:
dft2.head()

,ID,Age,IC,OC,Schooling,Timely Income,Married_Life,Cast,Hispanic,Sex,Full/Part,Gain,Loss,Stock,Tax Status,Summary,Weight,NOP,Citizen,Own/Self,Vet_Benefits,Weaks,WorkingPeriod
0,32701,57,1,44,Edu9,0,MS5,TypeA,HE,M,FB,0,0,0,J4,sum2,2087.06,1,Case1,0,2,52,94
1,105694,6,0,0,Edu4,0,MS3,TypeA,HA,F,FB,0,0,0,J1,sum4,979.61,0,Case1,0,0,0,94
2,139722,16,43,31,Edu14,0,MS3,TypeD,HA,F,FA,0,0,0,J1,sum1,2993.58,0,Case1,0,2,0,95
3,196082,56,0,0,Edu16,0,MS2,TypeA,HA,F,FA,0,0,0,J1,sum2,2216.33,0,Case1,0,2,0,95
4,82783,6,0,0,Edu4,0,MS3,TypeA,HA,M,FB,0,0,0,J1,sum4,429.67,0,Case1,0,0,0,95


In [32]:
dft2 = dft2.replace({"?": np.nan})

In [33]:
dft2 = dft2.fillna(dft2.mode().iloc[0])

In [34]:
dft2.isnull().values.any()

False

In [35]:
dft2.head()

,ID,Age,IC,OC,Schooling,Timely Income,Married_Life,Cast,Hispanic,Sex,Full/Part,Gain,Loss,Stock,Tax Status,Summary,Weight,NOP,Citizen,Own/Self,Vet_Benefits,Weaks,WorkingPeriod
0,32701,57,1,44,Edu9,0,MS5,TypeA,HE,M,FB,0,0,0,J4,sum2,2087.06,1,Case1,0,2,52,94
1,105694,6,0,0,Edu4,0,MS3,TypeA,HA,F,FB,0,0,0,J1,sum4,979.61,0,Case1,0,0,0,94
2,139722,16,43,31,Edu14,0,MS3,TypeD,HA,F,FA,0,0,0,J1,sum1,2993.58,0,Case1,0,2,0,95
3,196082,56,0,0,Edu16,0,MS2,TypeA,HA,F,FA,0,0,0,J1,sum2,2216.33,0,Case1,0,2,0,95
4,82783,6,0,0,Edu4,0,MS3,TypeA,HA,M,FB,0,0,0,J1,sum4,429.67,0,Case1,0,0,0,95


In [36]:
ids=dft2["ID"]
x_test=dft2.drop(columns=["ID"])

In [37]:
x_test = pd.get_dummies(x_test, columns=['Schooling', 'Married_Life', 'Cast',
        'Hispanic', 'Sex', 'Full/Part', 'Tax Status',
        'Summary', 'NOP', 'Citizen', 'Own/Self', 'Vet_Benefits',])

In [38]:
x_test.head()

,Age,IC,OC,Timely Income,Gain,Loss,Stock,Weight,Weaks,WorkingPeriod,Schooling_Edu1,Schooling_Edu10,Schooling_Edu11,Schooling_Edu12,Schooling_Edu13,Schooling_Edu14,Schooling_Edu15,Schooling_Edu16,Schooling_Edu17,Schooling_Edu2,Schooling_Edu3,Schooling_Edu4,Schooling_Edu5,Schooling_Edu6,Schooling_Edu7,Schooling_Edu8,Schooling_Edu9,Married_Life_MS1,Married_Life_MS2,Married_Life_MS3,Married_Life_MS4,Married_Life_MS5,Married_Life_MS6,Married_Life_MS7,Cast_TypeA,Cast_TypeB,Cast_TypeC,Cast_TypeD,Cast_TypeE,Hispanic_HA,Hispanic_HB,Hispanic_HC,Hispanic_HD,Hispanic_HE,Hispanic_HF,Hispanic_HG,Hispanic_HI,Hispanic_HJ,Sex_F,Sex_M,Full/Part_FA,Full/Part_FB,Full/Part_FC,Full/Part_FD,Full/Part_FE,Full/Part_FF,Full/Part_FG,Full/Part_FH,Tax Status_J1,Tax Status_J2,Tax Status_J3,Tax Status_J4,Tax Status_J5,Tax Status_J6,Summary_sum1,Summary_sum2,Summary_sum3,Summary_sum4,Summary_sum5,Summary_sum6,Summary_sum7,Summary_sum8,NOP_0,NOP_1,NOP_2,NOP_3,NOP_4,NOP_5,NOP_6,Citizen_Case1,Citizen_Case2,Citizen_Case3,Citizen_Case4,Citizen_Case5,Own/Self_0,Own/Self_1,Own/Self_2,Vet_Benefits_0,Vet_Benefits_1,Vet_Benefits_2
0,57,1,44,0,0,0,0,2087.06,52,94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
1,6,0,0,0,0,0,0,979.61,0,94,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
2,16,43,31,0,0,0,0,2993.58,0,95,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
3,56,0,0,0,0,0,0,2216.33,0,95,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,6,0,0,0,0,0,0,429.67,0,95,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0


In [39]:
x_test = x_test[sel_features]

In [40]:
x_test.head()

,Cast_TypeC,Cast_TypeD,Cast_TypeE,Married_Life_MS4,Schooling_Edu13,NOP_6,Loss,Schooling_Edu1,Own/Self_1,Schooling_Edu12,Schooling_Edu14,Schooling_Edu3,Schooling_Edu16,Hispanic_HA,Sex_M,Schooling_Edu15,Tax Status_J1,Schooling_Edu9,Tax Status_J5,Schooling_Edu5,Summary_sum3,Schooling_Edu17,Schooling_Edu6,Timely Income,Weaks,Stock,Gain,IC,OC,Age
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,52,0,0,1,44,57
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6
2,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,43,31,16
3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,56
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6


In [41]:
len(x_test.columns)

30

In [42]:
y_test = abc2.predict(x_test)

In [43]:
len(y_test)

99523

In [44]:
dicty= {"ID" : list(ids), "Class" : list(y_test)}

In [45]:
sub=pd.DataFrame.from_dict(dicty)

In [46]:
sub=sub[['ID', 'Class']]

In [47]:
sub.head()

,ID,Class
0,32701,0
1,105694,0
2,139722,0
3,196082,0
4,82783,0


In [48]:
sub.to_csv("last_submission.csv", index=False)

In [49]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
  
  csv = df.to_csv(index=False)
  b64 = base64.b64encode(csv.encode())
  payload = b64.decode()
  html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
  html = html.format(payload=payload,title=title,filename=filename)
  return HTML(html)

create_download_link(sub)